## Performance comparison parquet vs (csv, pickle)


In [2]:
import time


def timer(func):
    def wrapper(*args, **kwargs):
        begin = time.time()
        val = func(*args, **kwargs)
        end = time.time()
        print(f"The time taken in {func.__name__}: {end - begin :.2f} seconds")
        return val

    return wrapper

In [3]:
import os
import sys

root = os.path.dirname(os.path.abspath("."))
sys.path.append(root)

from pathlib import Path

import numpy as np

import predikit as pk

csv_file = Path("./sample_data/Flights 1m.csv")

data = {
    "Name": ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
    "Age": [np.nan, 23, 25, 50, np.nan, 50],
    "Credit": [np.nan, 400, np.nan, 50, 200, np.nan],
}

# df = pk.DataFrameParser(csv_file, index_col=0)  # fixme
df = pk.DataFrameParser(data)

# dp = pk.DataPrepare(clean_missing=True, clean_indicator=True, verbose=True)
# dp = pk.MissingValuesProcessor(
#     strategy=pk.MissingValueStrategy.MEAN,
#     add_indicator=True,
#     verbose=True,
# )

# dp.fit_transform(df, columns=["Age", "Credit"])

display(df)

,Name,Age,Credit
0,John,NaN,NaN
1,Andrea,23.0,400.0
2,Rose,25.0,NaN
3,Linda,50.0,50.0
4,Peter,NaN,200.0
5,Meg,50.0,NaN


In [4]:
path = "./sample_data/weatherAUS.csv"


cols = ["Evaporation", "Sunshine", "Cloud3pm"]


def init_df():
    df = pk.DataFrameParser(path)

    return df[cols]


df

,Name,Age,Credit
0,John,NaN,NaN
1,Andrea,23.0,400.0
2,Rose,25.0,NaN
3,Linda,50.0,50.0
4,Peter,NaN,200.0
5,Meg,50.0,NaN


# Predikit's MissingValueProcessor vs Sklearn's SimpleImputer


In [5]:
from time import time

from sklearn.impute import SimpleImputer

df = init_df()
display(df)

start = time()
si = SimpleImputer(strategy="most_frequent")
X = si.fit_transform(df)
df[cols] = X
end = time()
sklearn_time = end - start

df = init_df()

start = time()
mvp = pk.MissingValuesProcessor(strategy=pk.MissingValueStrategy.MODE)
mvp.fit_transform(df)
end = time()
predikit_time = end - start
print(f"From Sklearn: {sklearn_time:.3f} ms")
print(f"From Predikit: {predikit_time:.3f} ms")

speed_pct = (sklearn_time - predikit_time) / sklearn_time * 100

print(
    f"PrediKit's MissingValueProcessor is {speed_pct:.2f}% faster "
    "than Sklearn's SimpleImputer"
)

,Evaporation,Sunshine,Cloud3pm
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,2.0
3,NaN,NaN,NaN
4,NaN,NaN,8.0
...,...,...,...
142188,NaN,NaN,NaN
142189,NaN,NaN,NaN
142190,NaN,NaN,NaN
142191,NaN,NaN,NaN


From Sklearn: 0.028 ms
From Predikit: 0.019 ms
PrediKit's MissingValueProcessor is 32.72% faster than Sklearn's SimpleImputer


In [6]:
import timeit

# Define the setup code
setup_code = """
from sklearn.impute import SimpleImputer
import predikit as pk
import pandas as pd
dataset = "./sample_data/weatherAUS.csv"
cols = ["Evaporation", "Sunshine", "Cloud3pm"]
def init_df():
    df = pd.read_csv(dataset)
    return df[cols]
"""

# Define the code for SimpleImputer
si_code = """
df = init_df()
si = SimpleImputer(strategy=pk.MissingValueStrategy.MEAN)
X = si.fit_transform(df)
df[cols] = X
"""

# Define the code for MissingValuesProcessor
mvp_code = """
df = init_df()
mvp = pk.MissingValuesProcessor(strategy=pk.MissingValueStrategy.MEAN)
df = mvp.fit_transform(df)
"""

# Time the execution of the code
si_time = timeit.timeit(si_code, setup=setup_code, number=1)
mvp_time = timeit.timeit(mvp_code, setup=setup_code, number=1)

# Calculate the speed increase
speed_pct = (si_time - mvp_time) / si_time * 100

print(
    f"PrediKit's MissingValueProcessor is {speed_pct:.2f}% "
    "faster than Sklearn's SimpleImputer"
)

PrediKit's MissingValueProcessor is 11.38% faster than Sklearn's SimpleImputer


# Dask


In [7]:
# import dask.dataframe as dd

# df = dd.read_csv("./sample_data/Flights 1m.csv")

# df.head()

AttributeError: 'DataPrepare' object has no attribute 'clean_indicator'

AttributeError: 'DataPrepare' object has no attribute 'clean_indicator'

AttributeError: 'DataPrepare' object has no attribute 'clean_indicator'